<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Ecommerce-Churn-Assignment" data-toc-modified-id="Ecommerce-Churn-Assignment-1">Ecommerce Churn Assignment</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Data-description" data-toc-modified-id="Data-description-1.0.1">Data description</a></span></li><li><span><a href="#Initialising-the-SparkSession" data-toc-modified-id="Initialising-the-SparkSession-1.0.2">Initialising the SparkSession</a></span></li></ul></li><li><span><a href="#Task-4:-Model-Inference" data-toc-modified-id="Task-4:-Model-Inference-1.1">Task 4: Model Inference</a></span><ul class="toc-item"><li><span><a href="#The-best-feature-that-determines-Customer-churn-is-average-expenditure-by-customers-in-a-certain-category" data-toc-modified-id="The-best-feature-that-determines-Customer-churn-is-average-expenditure-by-customers-in-a-certain-category-1.1.1">The best feature that determines Customer churn is average expenditure by customers in a certain category</a></span></li><li><span><a href="#one-way-to-increase-Customer-purchase,-is-to-provide-some-discounts-to-the-products-that-the-customer-views" data-toc-modified-id="one-way-to-increase-Customer-purchase,-is-to-provide-some-discounts-to-the-products-that-the-customer-views-1.1.2">one way to increase Customer purchase, is to provide some discounts to the products that the customer views</a></span></li></ul></li></ul></li></ul></div>

# Ecommerce Churn Assignment

The aim of the assignment is to build a model that predicts whether a person purchases an item after it has been added to the cart or not. Being a classification problem, you are expected to use your understanding of all the three models covered till now. You must select the most robust model and provide a solution that predicts the churn in the most suitable manner. 

For this assignment, you are provided the data associated with an e-commerce company for the month of October 2019. Your task is to first analyse the data, and then perform multiple steps towards the model building process.

The broad tasks are:
- Data Exploration
- Feature Engineering
- Model Selection
- Model Inference

### Data description

The dataset stores the information of a customer session on the e-commerce platform. It records the activity and the associated parameters with it.

- **event_time**: Date and time when user accesses the platform
- **event_type**: Action performed by the customer
            - View
            - Cart
            - Purchase
            - Remove from cart
- **product_id**: Unique number to identify the product in the event
- **category_id**: Unique number to identify the category of the product
- **category_code**: Stores primary and secondary categories of the product
- **brand**: Brand associated with the product
- **price**: Price of the product
- **user_id**: Unique ID for a customer
- **user_session**: Session ID for a user


### Initialising the SparkSession

The dataset provided is 5 GBs in size. Therefore, it is expected that you increase the driver memory to a greater number. The m4.xlarge instance holds 16 GB memory and we are allocating 14 GB to the driver.

In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.driver.memory": "14000M"}
}

In [2]:
# installing required libraries
spark.sparkContext.getConf().get('spark.driver.memory')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1595397545612_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'14000M'

In [3]:
# loading the dataset from the parquet file
df = spark.read.parquet("s3://shiva-test-bucket-2/NOTEBOOK_1_V2.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Check if the dataframe is correctly loaded
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+-----------+-----------+----------+------------+-------------+--------------+-------------------------+-------------------------+------------------+----------------+--------------+----------------+---------------+-----------------+---------------+----------------+---------------+-----------------+-----+--------------------+
|  category_1| category_2|avg_expense|time_of_day|top_brands|is_purchased|day_of_week_d|product_view_d|event_per_session_count_d|sec_category_view_count_d|distinct_session_d|category_1_index|category_1_vec|category_2_index| category_2_vec|time_of_day_index|time_of_day_vec|top_brands_index| top_brands_vec|day_of_week_d_vec|label|            features|
+------------+-----------+-----------+-----------+----------+------------+-------------+--------------+-------------------------+-------------------------+------------------+----------------+--------------+----------------+---------------+-----------------+---------------+----------------+------

In [5]:
# exploring the dataframe - schema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = true)
 |-- avg_expense: double (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- top_brands: string (nullable = true)
 |-- is_purchased: integer (nullable = true)
 |-- day_of_week_d: double (nullable = true)
 |-- product_view_d: double (nullable = true)
 |-- event_per_session_count_d: double (nullable = true)
 |-- sec_category_view_count_d: double (nullable = true)
 |-- distinct_session_d: double (nullable = true)
 |-- category_1_index: double (nullable = true)
 |-- category_1_vec: vector (nullable = true)
 |-- category_2_index: double (nullable = true)
 |-- category_2_vec: vector (nullable = true)
 |-- time_of_day_index: double (nullable = true)
 |-- time_of_day_vec: vector (nullable = true)
 |-- top_brands_index: double (nullable = true)
 |-- top_brands_vec: vector (nullable = true)
 |-- day_of_week_d_vec: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vect

In [6]:
df = df.drop('is_purchased')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<hr>

## Task 4: Model Inference

- Feature Importance
- Model Inference
- Feature exploration

Based on the Precision and Recall of the 3 Models and considering the Time it took to fit these models, My Decision is to use Decision Trees, because
1. it Has a higher overall recall and higher Positive Class Recall than Logistic Regression model
2. The time it takes to fit DT/Logistic Regression model is very comparable so model with better performance is chosen
3. Perform Cross Validation for Random Forest was much longer with high chance of failure of the Cluster due to OOM error. And the "Best Parameter" which was obtained after Cross Validation does not perform as well as DT. 

In [7]:
# Split Data into train-test
df_train, df_test = df.randomSplit([0.7, 0.3], seed = 2020)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.ml.classification import DecisionTreeClassifier

# Fit Decision Tree on the best param obtained from Cross Validation
# maxDepth = 10
# maxBins = 100
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label',
                           maxDepth = 10, maxBins = 100)

# Fit the model
model = dt.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Transform test data to get predictions
predictions = model.transform(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Model Evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol = 'label',
                                             predictionCol = 'prediction',
                                             metricName = 'accuracy')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# model Accuracy
accuracy = evaluator.evaluate(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Get recall and precision
from pyspark.mllib.evaluation import MulticlassMetrics

preds_labels = predictions.select('prediction', 'label')
metrics = MulticlassMetrics(preds_labels.rdd.map(tuple))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Confusion matrix
print(metrics.confusionMatrix().toArray())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[3161227.   31042.]
 [  13793.   64160.]]

In [16]:
# Overall precision
precision = metrics.precision()
# Precision for label 0
precision_0 = metrics.precision(0)
# Precision for label1
precision_1 = metrics.precision(1)

print("Oveerall Precision = {}".format(precision))
print("Negavive class Precision = {}".format(precision_0))
print("Positive class Precision = {}".format(precision_1))

# overall recall
recall = metrics.recall()
# Recall for label0
recall_0 = metrics.recall(0)
# Recall for label1
recall_1 = metrics.recall(1)

print("Overall recall = {}".format(recall))
print("Negavive class recall = {}".format(recall_0))
print("Positive class recall = {}".format(recall_1))

#Calculate f1 score
f1Score = metrics.fMeasure()
# F1 score for label 0
f1_score_0 = metrics.fMeasure(0.0, beta = 1.0)
# F1 score for label 1
f1_score_1 = metrics.fMeasure(1.0, beta = 1.0)

print("Overall f1Score = {}".format(f1Score))
print("Negavive class fMeasure = {}".format(f1_score_0))
print("Positive class fMeasure = {}".format(f1_score_1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Oveerall Precision = 0.9862899216016527
Negavive class Precision = 0.9956557753966904
Positive class Precision = 0.6739354215247579
Overall recall = 0.9862899216016527
Negavive class recall = 0.9902758821390052
Positive class recall = 0.823060049003887
Overall f1Score = 0.9862899216016527
Negavive class fMeasure = 0.9929585416964739
Positive class fMeasure = 0.7410701394704167

In [19]:
# installing required libraries
spark.sparkContext.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
spark.sparkContext.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository
spark.sparkContext.install_pypi_package("numpy==1.16.5")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Feature Importance
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata['ml_attr']['attrs']:
        list_extract = list_extract + dataset.schema[featuresCol].metadata['ml_attr']['attrs'][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
ExtractFeatureImp(model.featureImportances, predictions, 'features').head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    idx                        name     score
3    94                 avg_expense  0.816215
0    91   event_per_session_count_d  0.075578
76   71       top_brands_vec_others  0.045443
1    92              product_view_d  0.029866
77   72      top_brands_vec_samsung  0.009249
5     0  category_1_vec_electronics  0.007649
8     3      category_1_vec_apparel  0.002827
79   74       top_brands_vec_xiaomi  0.001698
82   77         top_brands_vec_oppo  0.001358
85   80       top_brands_vec_lenovo  0.001158

### The best feature that determines Customer churn is average expenditure by customers in a certain category

### one way to increase Customer purchase, is to provide some discounts to the products that the customer views